# Pattern Matching vs. Unification

지금쯤이면 이미 수업에서 하스켈 코드가 포함된 강의 노트에서도 접해보고
또 과제를 하며 **패턴 매칭**(pattern matching)을 활용한 코드를 직접 작성해 보았을 것이다.
우리가 수업에서 쓰는 하스켈이나 또 OCaml 같은 함수형 언어에서 이런 패턴 매칭이 프로그래밍 언어의 기능으로 제공된다.

한편 앞으로 수업에서 사용할 Prolog라는 언어에서는 **일치화**(unification)이라는 기능이 제공되는데
패턴 매칭과 비슷한 점도 있기 때문에 이 두 개념을 연관지어 이해하면서 그 차이점을 확실히 파악할 수 있다면
앞으로 수업에서도 도움이 될 것이다.

여기서는 타입 검사 및 유추라는 맥락에서 패턴 매칭과 일치화에 대해 설명하겠다.

## 패턴 매칭 복습

In [37]:
data Ty = INT
        | Ty :-> Ty
        deriving (Show,Eq,Ord)
infixr 8 :-> -- 함수 기호가 일반적으로 오른쪽으로 묶이도록 표기하므로 오른편 결합 설정

하스켈에서 가장 기본적인 정의는 `변수 = 식` 형태로
왼편에 하나의 이름(또는 변수)을 쓰고 등호를 쓴 다음 오른편에 그 이름이 가질 값을 계산할 식을 쓰면 된다.

아래는 이와 같은 형태의 정의가 두 번 나타난다.

In [39]:
ty1 = INT :-> (INT :-> INT)
ty2 = (INT :-> INT) :-> INT

이렇게 정의된 `ty1`과 `ty2`의 값을 출력해 보면 다음과 같다.

In [40]:
ty1

INT :-> (INT :-> INT)

In [41]:
ty2

(INT :-> INT) :-> INT

하지만 하스켈과 같은 함수형 언어에서는 정의의 등호 왼편에 하나의 변수만 올 수 있는 것이 아니라
일반적으로 여러 개의 변수가 포함된 **패턴**이 올 수 있으며 여러 개의 변수를 포함하는 패턴을
활용해 정의하면 한꺼번에 여러 변수를 정의하는 것과 같은 효과를 얻을 수 있다. 즉 정의의 더
일반적인 형식은 `패턴 = 식`이다. 

패턴은 `INT`와 같이 그 자체로 데이타인 데이타 상수,
`:->`와 같이 몇 개의 인자를 받아 데이타를 만들어내는 데이타 생성자,
그리고 새로이 정의될 변수들로 구성된다.

패턴을 이용한 아래의 두 정의를 살펴보자.

In [45]:
dom1 :-> rng1 = ty1 -- INT :-> (INT :-> INT)
dom2 :-> rng2 = ty2 -- (INT :-> INT) :-> INT

In [46]:
dom1
rng1

INT

INT :-> INT

In [47]:
dom2
rng2

INT :-> INT

INT

여러 겹으로 된 패턴으로 더 많은 변수를 한꺼번에 정의할 수도 있다.

In [48]:
dom1 :-> (rd1 :-> rr1) = ty1 -- INT :-> (INT :-> INT)
(dd2 :-> rd2) :-> rng2 = ty2 -- (INT :-> INT) :-> INT

In [49]:
dom1
rd1
rd2

INT

INT

INT

In [50]:
dd2
rd2
rng2

INT

INT

INT

주의해야 할 점은 패턴 매칭이 실패하여 에러가 발생하며 프로그램이 비정상 종료될 수 있다는 점이다.

좀전에 원래 `ty1`을 사용해던 정의를 `ty1`대신에 `ty2`로 바꿔보자.

In [51]:
dom1 :-> (rd1 :-> rr1) = ty2  -- (INT :-> INT) :-> INT

하스켈은 게으른 계산법을 따르므로 정의하는 것만으로는 에러가 발생하지는 않는다.

그러나 위의 정의로 새로이 정의된 이름들 중 하나라도 값을 요구하면 아래와 같이 에러가 발생한다.

In [52]:
dom1

: 

에러가 발생하는 이유는 패턴 매칭의 오른쪽 부분인 `(rd1 :-> rr1)`의 구조에 `ty2`로부터 계산되는 값의 오른쪽 부분인 `INT`를 대응시킬 수 없기 때문이다.

실제로 에러가 발생하는 원인인 부분을 따로 떼어내서 좀더 작은 정의를 만들어 보면 그 이유가 더 확실히 드러나 보인다.

In [53]:
rd1 :-> rr1 = INT

In [54]:
rd1

: 

그렇기 때문에 어떤 데이터 타입의 값이 가질 수 있는 형태가 한 가지로 일정한 경우(예: `(int,int)`와 같은 순서쌍)를 제외하고는
대부분의 경우 패턴 매칭은 여러 가지 형태가 될 수 있는 경우를 모두 고려해야 하므로 `case ... of ...` 식으로 여러 가지
가능한 패턴을 처리하거나 여러 줄의 등식으로 정의되는 함수 정의에서 여러 가지 경우의 함수 인자 형태를 고려하는 방식으로 작성된다.
지금까지 우리 수업시간에 다룬 예제 코드나 과제에서도 패턴 매칭을 대부분 이런 방식으로 활용했음을 상기해 보라.

## 일치화
일치화(unification)은 패턴 매칭(pattern matching)의 기능을 포함하는 더 일반적인 매칭 방식이다.

이를 설명하기 위해 우선 패턴 매칭을 특정 프로그래밍 언어에서 제공하는 기능으로써가 아니라 좀더 일반적인 개념으로
나무구조에 대한 방정식을 푸는 것이라 생각해 보자.
방정식이라는 측면을 강조하기 위해 변수 이름을 방정식에 흔히 쓰는 `x`와 `y`를 써서 아래와 같은 패턴 매칭을 한다고 생각해 보자.

$x\;\text{:->}\;y ~=~ \texttt{INT}\;\text{:->}\;(\texttt{INT}\;\text{:->}\;\texttt{INT})$

그리고 위 방정식의 해를 다음과 같이 방정식에 나타나는 변수를 값들로 대응시키는 치환(substitution)이라는 개념으로
이해하여 다음과 같이 표현할 수 있다.

$\{\texttt{x} \mapsto \texttt{INT},\;\texttt{y}\mapsto \texttt{INT}\;\text{:->}\;\texttt{INT} \}$

패턴 매칭을 표현하는 방정식의 두 가지 특징은 다음과 같다.
 1. 우선 모든 변수는 등식의 한쪽에만 나타난다.
 2. 방정식의 측면으로만 보면 꼭 그래야 하는 것은 아니지만,
 일반적으로 프로그래밍 언어의 기능으로 제공될 때에는 변수가 반복해서 등장하는 것을 허용하지 않는다.

두 번째 특징은 $x\;\text{:->}\;x ~=~ \cdots$ 처럼 방정식에 똑같은 변수가 두 번 이상 나타나서는 안된다는 것을 의미한다.
패턴 매칭이 프로그래밍 언어에서 여러 번의 변수 정의를 한꺼번에 하기 편하게 해주는 효과가 있는데
똑같은 변수가 두 번 이상 반복된다면 두 번 초기화가 일어나는 부자연스러운 의미로 해석될 수 있기 때문이다.

일치화(unification)는 방금 정리한 두 가지 특징에 구애받지 않는 일반적인 나무구조에 대한 방정식의 풀이 방법을 말한다.
즉, 방정식에서 양쪽 어디에나 변수 올 수 있으며 같은 변수가 등식에 두 번 이상 등장해도 된다는 것이다.

예컨대, 다음과 같은 방정식을 일치화로 풀 수 있다.

$(\texttt{INT}\;\text{:->}\;x)\;\text{:->}\;\texttt{INT} ~=~ y\;\text{:->}\;x$

우선 양쪽 다 화살표($\text{:->}$) 구조이므로 화살표의 왼편은 왼편끼리 오른쪽은 오른편끼리 일치화시키면 된다.
즉 위에서 본 하나의 방정식을 해는 아래와 같은 두 개의 방정식으로 이루어진 연립방정식의 해와 같다.

$\displaystyle
\begin{array}{l}
\texttt{INT}\;\text{:->}\;x ~=~ y
\\
\texttt{INT} ~=~ x
\end{array}$

위 연립방정식의 해는 $\{ x\mapsto \texttt{INT},\; y\mapsto\texttt{INT}\;\text{:->}\;\texttt{INT} \}$이다.

Prolog는 언어의 기본 기능으로 일치화를 제공한다. 즉 위와 같은 방정식을 자동으로 풀어준다.
지금까지는 하스켈 문법을 기준으로 변수를 소문자로 그리고 구체적인 값에 해당하는 데이타 상수 및 생성자를 대문자 시작하는 단어들로 표기했다.
하지만 Prolog는 문법이 이와는 대조적으로 변수를 대문자로 그리고 구체적인 값에 해당하는 구조를 소문자로 시작한다는 점에 유의하자.
Prolog 문법으로는 방금 살펴본 방정식을 아래와 같이 표현할 수 있다.

```
((int -> X) -> int) = (Y -> X).
```

참고로 `->`는 Prolog에 미리 정의되어 있는 연산자이며
이를 데이타로 사용할 경우에는 괄호로 묶어서 써주는 것이 좋다. 또한 Prolog에서는 소문자로 시작하는
`int`와 같은 상수를 아톰(atom)이라 하며 미리 선언하지 않고 바로 사용할 수 있다.

이제 실제로 [Tau-Prolog](http://tau-prolog.org) 홈페이지에서 위와 같은 방정식의 풀이가 자동으로 계산되는 것을 살펴보자. 